## Prepare Referece files using Word2Vec for retrieving attributes


In [13]:
import gensim.downloader as api
import numpy as np

wv = api.load("word2vec-google-news-300")

data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"
output_dir = data_dir + "/word2vec/"

def read_file(path):
    with open(path) as fp:
        lines = fp.read().splitlines()
    return lines

In [2]:
# test
print(wv.most_similar(positive=["king", "woman"], negative=["man"], topn=5))

[('queen', 0.7118193507194519), ('monarch', 0.6189674735069275), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377322435379028)]


In [11]:
# Extract attr from train corpus

import itertools

data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"



train0_processed = read_file(data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_0_all_attrs.txt") # training data with content and attributes seperation
train1_processed = read_file(data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_1_all_attrs.txt") # training data with content and attributes seperation

def get_train_attrs(text):
    return text.split("<CON_START>")[0].replace("<ATTR_WORDS>","").strip().split()

# Fatch attributes from the training data
attrs_neg = [get_train_attrs(x) for x in train0_processed]
attrs_pos = [get_train_attrs(x) for x in train1_processed]

print(list(itertools.chain(*attrs_pos))[:4])
print(list(itertools.chain(*attrs_neg))[:4])

print("laaaaaaaaaaaaaaaaa" in wv.index_to_key)
print("go" in wv.index_to_key)

filterd_list_1 = set(filter(lambda x: x in list(wv.index_to_key), list(itertools.chain(*attrs_pos))))
filterd_list_0 = set(filter(lambda x: x in list(wv.index_to_key), list(itertools.chain(*attrs_neg))))

with open(output_dir+"filtered_pos_1.txt", 'w') as outfile:
    for word in filterd_list_1:
            outfile.write(word +"\n")

with open(output_dir+"filtered_neg_0.txt", 'w') as outfile:
    for word in filterd_list_0:
            outfile.write(word +"\n")

['excellent', 'superb', 'daily', 'really']
['sadly', 'so', 'run', 'mill']
False
True


In [32]:



train_corpus_pos_path = data_dir + "/word2vec/" + "filtered_pos_1.txt"
train_corpus_neg_path = data_dir + "/word2vec/" + "filtered_neg_0.txt"

# Source https://gist.githubusercontent.com/kylemcdonald/9bedafead69145875b8c/raw/8236702165155cf1fa3e7aa38122380d3b95ac88/antonyms.txt
train_corpus_pairs_path = data_dir + "/word2vec/" + "antonyms.txt"

#train_corpus_pos = read_file(train_corpus_pos_path)
#train_corpus_neg = read_file(train_corpus_neg_path)
train_corpus_pairs = read_file(train_corpus_pairs_path)
print(train_corpus_pairs[4].split(",")[0])
print(train_corpus_pairs[3].split(",")[1])


admit
inaccurate


In [49]:
#positive_words = train_corpus_pos
#negative_words = train_corpus_neg
positive_words = []
negative_words = []

for pair in train_corpus_pairs:
    positive_words.append(pair.split(",")[0])    
    negative_words.append(pair.split(",")[1])    

positive_words = ["good", "great", "beautiful", "positive", "delicious"]
negative_words = ["bad", "worse", "awful", "negative", "disgusting"]

v_positive = np.average(np.array(list(map(lambda x: wv[x], positive_words))), axis=0)

v_negative = np.average(np.array(list(map(lambda x: wv[x], negative_words))), axis=0)

print(positive_words[:4])
print(negative_words[:4])

['good', 'great', 'beautiful', 'positive']
['bad', 'worse', 'awful', 'negative']


In [40]:
from typing import List


def retrieve_antonyms(attributes: List[str], to_positive: bool = True) -> List[str]:
    transferred_attributes = list()
    for attribute in attributes:
        if to_positive:
            v_query = wv[attribute] + v_positive - v_negative
        else:
            v_query = wv[attribute] - v_positive + v_negative
        antonym = wv.most_similar(positive=[v_query], topn=1)
        transferred_attributes.append(antonym[0][0])
    return transferred_attributes

In [53]:
# test ['positive', 'noisy', 'hungry']
print(retrieve_antonyms(["negative"]))

['positive']


In [ ]:
def read_file(path):
    with open(path) as fp:
        lines = fp.read().splitlines()
    return lines

In [ ]:
def create_file(input_file, output_file, to_positive=True):
    with open(output_file, "w") as out_fp:
        input_lines = read_file(input_file)
        for input_line in input_lines:
            # each input line looks like this:
            # [ATTRS]delicious<CONT_START>I never tasted such <REPLACE> soup.<END>
            splits = input_line.split("<CONT_START>")
            attributes = splits[0].remove("[ATTRS]")
            antonyms = retrieve_antonyms(attributes, to_positive)
            output_line = "[ATTRS]" + " ".join(antonyms) + splits[1]
            out_fp.write(output_line)

In [ ]:
data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"

reference_0 = data_dir + "/processed_files_with_bert_with_best_head/reference_0.txt"
reference_1 = data_dir + "/processed_files_with_bert_with_best_head/reference_1.txt"
out_reference_0 = data_dir + "/processed_files_with_bert_with_best_head/word2vec/reference_0.txt"
out_reference_1 = data_dir + "/processed_files_with_bert_with_best_head/word2vec/reference_1.txt"

create_file(reference_0, out_reference_0, True)
create_file(reference_1, out_reference_1, False)

In [54]:
def read_file(path):
    with open(path) as fp:
        lines = fp.read().splitlines()
    return lines

def get_attrs(text):
    return text.split("<CON_START>")[0].replace("<ATTR_WORDS>","").strip().split()

In [58]:
''' Load Atts '''
data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"

ref0_own_Att = read_file(data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/reference_0.txt") # Reference data for delete_generate model
ref1_own_Att = read_file(data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/reference_1.txt") # Reference data for delete_generate model

ref_out_path = data_dir + "/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/"

print(get_attrs(ref0_own_Att[1]))

ref0_att_antonyms = []
ref1_att_antonyms = []

for atts in ref0_own_Att:
    att_antonyms = []
    for word in get_attrs(atts):
        antonym = None
        if word in list(wv.index_to_key):
            antonym = retrieve_antonyms([word], True)
        if antonym != None:
            att_antonyms.append(antonym)
    ref0_att_antonyms.append(att_antonyms)

for atts in ref1_own_Att:
    att_antonyms = []
    for word in get_attrs(atts):
        antonym = None
        if word in list(wv.index_to_key):
            antonym = retrieve_antonyms([word], False)
        if antonym != None:
            att_antonyms.append(antonym)
    ref1_att_antonyms.append(att_antonyms)


print(ref0_att_antonyms[:10])
print(ref1_att_antonyms[:10])

['definitely', 'venue']
[[['worse'], ['worse']], [['definitely'], ['venue']], [['tasted'], ['watered']], [['said'], ['disappeared']], [['ca'], ['pharmacy']], [['left'], ['took']], [['great'], ['very']], [['definitely'], ['disappointed']], [['owner'], ['heard']], [['sucks'], ['too']]]
[[['small'], ['make']], [['enjoying'], ['terrible']], [['affordable'], ['bad']], [['sandwich'], ['loved']], [['signed'], ['got']], [['definitely'], ['recommend']], [['highly'], ['recommend']], [['otherwise'], ['terrible']], [['bad'], ['bad']], [['oh'], ['geek']]]


In [59]:
with open(ref_out_path+"reference_word2vec_0.txt", 'w') as outfile:
    for index,words in enumerate(ref0_att_antonyms):
        if len(words) > 0:
            outfile.write("<ATTR_WORDS> "+ " ".join([w[0] for w in words])+ " <CON_START>" +ref0_own_Att[index].split("<CON_START>")[1] +"\n")
        else: 
            outfile.write("<ATTR_WORDS> "+ "<CON_START>" +ref0_own_Att[index].split("<CON_START>")[1] +"\n")


with open(ref_out_path+"reference_word2vec_1.txt", 'w') as outfile:
    for index,words in enumerate(ref1_att_antonyms):
        if len(words) > 0:
            outfile.write("<ATTR_WORDS> "+ " ".join([w[0] for w in words])+ " <CON_START>" +ref1_own_Att[index].split("<CON_START>")[1] +"\n")
        else: 
            outfile.write("<ATTR_WORDS> " +"<CON_START>" +ref1_own_Att[index].split("<CON_START>")[1] +"\n")
        

TypeError: sequence item 0: expected str instance, list found

In [60]:
print(ref0_att_antonyms)
print(ref1_att_antonyms)



[[['worse'], ['worse']], [['definitely'], ['venue']], [['tasted'], ['watered']], [['said'], ['disappeared']], [['ca'], ['pharmacy']], [['left'], ['took']], [['great'], ['very']], [['definitely'], ['disappointed']], [['owner'], ['heard']], [['sucks'], ['too']], [['sit'], ['slow']], [['charge'], ['small']], [['no'], ['sorry']], [['said'], ['sit']], [['empty'], ['no']], [['ok'], ['bland']], [['average'], ['food']], [['staffed'], ['primarily']], [['cooked'], ['crunchy']], [['blue'], ['cheese']], [['pad'], ['tasted']], [['said']], [['store'], ['needs']], [['bread']], [['complained'], ['polite']], [['happy']], [['shape'], ['dry']], [['associates'], ['program']], [['lacking']], [['anyway'], ['got']], [['sales'], ['great']], [['not'], ['good']], [['so'], ['card']], [['just'], ['delivery']], [["n't"], ['let']], [['me'], ['lied']], [['beans'], ['burro']], [['tried'], ['advantage']], [["'ve"], ['sent']], [['rude'], ['shitty']], [['really'], ['need']], [['skipped'], ['went']], [['cold']], [['yes']